<a href="https://colab.research.google.com/github/aminehd/AminehTechnicalWritings/blob/main/SamplingStatsAndDistros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Outline
- Introduction (Purpose + scope)
- Body
  - sample summary statistics and examples (coefficients of fitted model)
  - load data and showcase
  - Sampling distribution of summary statistics. (use sample mean as example)
  - Expected value of summary statistics
  - Variance of summary statistic
  - Variance within a sample vs variance of a given stat over samplings
  - Standrad Error of stats and confidence interval. (SE(beta0))
  - F-test (within groups variance vs between group variance)
- Conclusion:

## Git setup and data load

In [1]:
import google.colab
repo_path = 'AminehTechnicalWritings'
!git -C $repo_path pull origin || git clone https://github.com/aminehd/AminehTechnicalWritings/ $repo_path
local_clone = '/content/' + repo_path
!git config --global user.email "amineh.dadsetan@gmail.com"
!git config --global user.name "Amineh Dadsetan"

fatal: cannot change to 'AminehTechnicalWritings': No such file or directory
Cloning into 'AminehTechnicalWritings'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 165 (delta 91), reused 139 (delta 79), pack-reused 3
Receiving objects: 100% (165/165), 3.12 MiB | 8.72 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [3]:
import numpy as np
files = np.load(local_clone + '/population.npz')
# sample = np.load(local_clone + '/sample.npy'
population = files['arr_0']
population.shape

(300000, 2)

### Summary Statistic

Previously we explained the concept of population and the concept of sample. Basically we have one population (e.g., population of American people) but we can sample a subset in many ways (e.g., sample of America people under age 18, sample of American people living in new your, etc).

In fact if N is population size, there is $2^N$ samples.

Now a statistic or measure can be defined on the whole population as well as any given sample. A statistic or a measure in mathematic is a generalization of concepts such as lenght or heigh for a given set. Since samples and the population are sets, we can define such things on them. A well known one is mean. Thus you can think of population mean or any given sample mean.

So far it was fairly easy to explain things. But it gets a bit trickier when we wanna talk the distribution of such measures over allll samples and compare them with the value of measure on the poulation.

The motivation behind sampling is that population is too big and it is virtually impossible to calculate the mean for the whole population. Let's say as a minister of something you need the average age of your population.  

Let's look at the population mean and sample mean for the data set we have created in previous notebooks.There are only 30 distinct x values in the pupolation data. Let pick a subset of such distinct x.




In [9]:
print(population[:, 0])
len(population[:,0])
distinct_xs = np.unique(population[:, 0])
len(distinct_xs)
x_samples = distinct_xs[np.linspace(0, len(distinct_xs)-1, 10).astype(int)]
x_samples

[ 1.  1.  1. ... 10. 10. 10.]


array([ 1.        ,  1.93103448,  2.86206897,  3.79310345,  4.72413793,
        5.96551724,  6.89655172,  7.82758621,  8.75862069, 10.        ])

We sampled 10 distinct x and stored them in `x_samples`. For each x there are several responses in the data set that are spaced closely around their average. Thus we can have several sampling with given `x_samples`. Let's pick one such and plot it:
Next thing is you wanna randomly pick y's that are response to these x's. Not easiest to implement now. think of abstraction HERE!!!



https://en.wikipedia.org/wiki/Sampling_distribution
start with mu but get to beta_i HERE!!!

The concept of unbiased estimator pops up one more time. HERE!!!


Then we look at expected value of sample statistices over multiple sampling. Also we look at variance of stuff within sample, and variance of sample stat over several samplings. This indeed is an intersting case when sample stat is defined to be the mean of sample. HERE!!!




notebook we try to look at the model coefficients ( 𝛽𝑖 's ) as summary statistics defined on samples. You can also think of them as measures since samples are subsets.



These can help understand concept of Standard Error and F-test better.